In [0]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
from google.colab import drive

drive.mount('./now1')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at ./now1


In [3]:
# 구글드라이브의 내드라이브 -> kaggle_study 폴더 생성
cd now1/My Drive/kaggle_study/

/content/now1/My Drive/kaggle_study


In [4]:
trn= pd.read_csv('./kaggle_data/train_ver2.csv')
tst= pd.read_csv('./kaggle_data/test_ver2.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (5,8,11,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [5]:
print(trn.shape)
print(tst.shape)

(13647309, 48)
(929615, 24)


# Santander_Product_Recommendation(2019.06.08)
_____

산탄데르 상품 추천 경진대회
- 목표: <br>
    * 지난달 고객이 보유하고 있지 않는 금융상품 중 이번달에 신규로 구매하게 되는 제품을 예측하는 것.<br>
    * 캐글에 제출해야하는 값은 고객이  16-05-28시점에 보유하고 있지 않은 상품중, 16-06-28에 구매할 것으로 예측되는 제품 상위 7개이다. (24개 중 7개)

- 특이사항<br>
    * 제공되는 데이터는 A라는 시간에 B라는 고객이 C라는 제품을 보유하고 있다는 정보로 이루어져 있다. 모델 학습에는 신규구매 건만 별도로 추출하여 학습데이터로 사용한다.  (1100만개 -> 20만개)

-----------------
* 코랩 사양
    * CPU : Intel Xeon 2.2GHz
    * RAM : 13GB
    *저장공간 : 33GB
    * GPU : Nvidia Tesla K80 
* Colab에서 실행하기 위한 메모리 절약팁
    1. 각 특성변수에 대해 적합한 타입을 지정한다.
       * 변수의 타입은 은 최대한 int형으로 변환
        -  int8 : -128~127
        -  int16: -32768 ~ 32767
        -  int32 : -2147483648 ~ 2147483647
    2. EDA로 파악된 필요없는 특성은 삭제한다.
    3. read_csv로 불러올때 dtype옵션을 설정한다.
        - https://www.dataquest.io/blog/pandas-big-data/
        - https://honingds.com/blog/pandas-read_csv/
---------

#1. 데이터 전처리

- 데이터 클리닝: 결측값/이상값 처리
- 카테고리 변수-> 라벨 인코딩
- 타입지정-> 메모리 사용을 최소화


In [6]:
# 24개 상품의 결측값은 0(미보유)으로 처리
prods= trn.columns[24:].tolist()
trn[prods]= trn[prods].fillna(0).astype(np.int8)

# 24개 상품 중 하나도 보유하지 않는 고객 데이터는 삭제한다. 왜?
# 가입만 해놓은 고객들->필요없는 정보
no_product= trn[prods].sum(axis=1) == 0 # 행에 대해서 모두 합한다음 0인지 아닌지
trn= trn[~no_product]
print(trn.shape)


(11091070, 48)


In [0]:
# 테스트셋에 24개 상품 컬럼을 생성한다.
for col in trn.columns[24:]:
    tst[col]= np.int8(0)    

# 학습셋과 테스트셋을 합친다. (추후 과정을 한꺼번에 처리)
df= pd.concat([trn, tst], axis=0) #  axis의 의미, 행방향으로 합친다. (같은 컬럼을 기준으로)

In [0]:
df.reset_index(inplace= True)
del df['index']

In [9]:
print(trn.shape)
print(tst.shape)
print(df.shape)

# (11091070, 48)
# (929615, 48)
# (12020685, 48)

del trn
del tst

(11091070, 48)
(929615, 48)
(12020685, 48)


In [10]:
categorical_cols= ['ind_empleado', 'pais_residencia', 'sexo', 
                   'tiprel_1mes', 'indresi', 'indext', 'conyuemp', 'canal_entrada',
                   'indfall', 'tipodom', 'nomprov', 'segmento']

for col in categorical_cols:
    print(col, ':', df[col].unique())

ind_empleado : ['N' nan 'A' 'B' 'F' 'S']
pais_residencia : ['ES' nan 'CA' 'CH' 'CL' 'IE' 'AT' 'NL' 'FR' 'GB' 'DE' 'DO' 'BE' 'AR' 'VE'
 'US' 'MX' 'BR' 'IT' 'EC' 'PE' 'CO' 'HN' 'FI' 'SE' 'AL' 'PT' 'MZ' 'CN'
 'TW' 'PL' 'IN' 'CR' 'NI' 'HK' 'AD' 'CZ' 'AE' 'MA' 'GR' 'PR' 'RO' 'IL'
 'RU' 'GT' 'GA' 'NO' 'SN' 'MR' 'UA' 'BG' 'PY' 'EE' 'SV' 'ET' 'CM' 'SA'
 'CI' 'QA' 'LU' 'PA' 'AU' 'BY' 'BO' 'KE' 'SG' 'HR' 'MD' 'SK' 'TR' 'AO'
 'CU' 'EG' 'ZA' 'DK' 'UY' 'TH' 'DZ' 'JP' 'NG' 'PK' 'TG' 'KR' 'RS' 'VN'
 'PH' 'KW' 'GQ' 'NZ' 'MM' 'KH' 'GI' 'SL' 'GN' 'GW' 'OM' 'CG' 'LV' 'LT'
 'ML' 'MK' 'HU' 'IS' 'LY' 'BA' 'CF' 'KZ' 'CD' 'BZ' 'BM' 'MT' 'DJ' 'JM'
 'TN' 'LB' 'GH' 'GE' 'ZW' 'GM']
sexo : ['H' 'V' nan]
tiprel_1mes : ['A' 'I' nan 'P' 'R' 'N']
indresi : ['S' nan 'N']
indext : ['N' 'S' nan]
conyuemp : [nan 'N' 'S']
canal_entrada : ['KHL' 'KHE' 'KHD' 'KFA' 'KFC' 'KAT' nan 'KAZ' 'RED' 'KHC' 'KHK' 'KGN'
 'KHM' 'KHO' 'KDH' 'KEH' 'KAD' 'KBG' 'KGC' 'KHF' 'KFK' 'KHA' 'KAF' 'KGX'
 'KFD' 'KAG' 'KFG' 'KAB' 'KCC' 'KAE' 'KAH' '

In [0]:
# 카테고리 변수를 라벨 인코딩한다.
for col in categorical_cols:
    df[col], _ = df[col].factorize(na_sentinel= -99) 

    if col not in ['pais_residencia', 'canal_entrada']:
        df[col]= df[col].astype(np.int8) # 라벨링 후 int8타입으로 설정
        
df['pais_residencia']= df['pais_residencia'].astype(np.int16)
df['canal_entrada']= df['canal_entrada'].astype(np.int16)

In [12]:
for col in categorical_cols:
    print(col, ':', df[col].unique())

ind_empleado : [  0 -99   1   2   3   4]
pais_residencia : [  0 -99   1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88
  89  90  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106
 107 108 109 110 111 112 113 114 115 116 117]
sexo : [  0   1 -99]
tiprel_1mes : [  0   1 -99   2   3   4]
indresi : [  0 -99   1]
indext : [  0   1 -99]
conyuemp : [-99   0   1]
canal_entrada : [  0   1   2   3   4   5 -99   6   7   8   9  10  11  12  13  14  15  16
  17  18  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34
  35  36  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52
  53  54  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70
  71  72  73  74

In [0]:
# 수치형 변수의 특이값과 결측값을 -99로 대체하고 정수형으로 변환한다.

df['age'].replace(' NA', -99, inplace= True)
df['age'].fillna(-99, inplace= True)
df['age']= df['age'].astype(np.int8)

df['antiguedad'].replace(['     NA', -999999], -99, inplace= True)
df['antiguedad'].fillna(-99, inplace= True)
df['antiguedad'] = df['antiguedad'].astype(np.int32)

df['renta'].replace('         NA', -99, inplace= True)
df['renta'].fillna(-99, inplace= True)
df['renta'] = df['renta'].astype(float).astype(np.int32)

df['indrel_1mes'].replace('P', 5, inplace= True)
df['indrel_1mes'].fillna(-99, inplace= True)
df['indrel_1mes']= df['indrel_1mes'].astype(float).astype(np.int8)

for col in ['ind_nuevo', 'indrel', 'tipodom', 'cod_prov', 'ind_actividad_cliente']:
    df[col] = df[col].fillna(-99).astype(np.int8)


In [0]:
#0 fecha_dato                object -> 문자,T: 날짜 2016-06-28
#1 ncodpers                  int64  -> 숫자,N: 고객고유식별번호 1170544
#2 ind_empleado              object -> 문자,C: 고용상태 (A,B,F,N,P 등) 
#3 pais_residencia           object -> 문자,C: 고객의 거주국가 ES,AD 등
#4 sexo                      object -> 문자,C: 성별 V,H
#5 age                       object -> 문자,N: 나이 1~164
#6 fecha_alta                object -> 문자,T: 은행 첫 계약날짜 2016-06-28
#7 ind_nuevo                 float64-> 숫자,N: 신규고객지표(6개월 이내 신규고객일경우: 1) 0/1
#8 antiguedad                object -> 문자,N: 은행 거래누적기간(월) 0~-999999, '35'
#9 indrel                    float64-> 숫자,N: 고객등급 1,99
#10 ult_fec_cli_1t           object -> 문자,T: 1등급고객으로서 마지막날짜 2016-06-28
#11 indrel_1mes              object -> 문자,N: 월초기준고객등급(1,2,P,3,4)
#12 tiprel_1mes              object -> 문자,C: 월초기준고객관계유형(A,I,P,R)
#13 indresi                  object -> 문자,C: 거주지표(거주국가와 은행국가 동일여부, S/N)
#14 indext                   object -> 문자,C: 외국인지표(고객의 태어난 국가와 은행국가 동일여부, S/N)
#15 conyuemp                 object -> 문자,C: 배우자지표(1: 은행직원을 배우자로 둔 고객, S/N)
#16 canal_entrada            object -> 문자,C: 고객유입채널(3, 4, KAT, KHD 등)
#17 indfall                  object -> 문자,C: 고객사망여부(S/N)
#18 tipodom                  float64-> 숫자,C: 주소유형(1:primary address)
#19 cod_prov                 float64-> 숫자,N: 지방코드(주소기반), 1~52
#20 nomprov                  object -> 문자,C: 지방이름, MADRID
#21 ind_actividad_cliente    float64-> 숫자,N: 활동성지표(1,2) 0/1
#22 renta                    float64-> 숫자,N: 가구총수입 42341.3
#23 segmento                 object->  문자,C: 고객분류(01,02,03) 01-TOP

#########################################################
# 상품 24~48 (모두,0/1/nan)
# **ind_ahor_fin_ult1        int64
# **ind_aval_fin_ult1        int64
# **ind_cco_fin_ult1         int64
# **ind_cder_fin_ult1        int64
# 이하 동일


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12020685 entries, 0 to 12020684
Data columns (total 48 columns):
fecha_dato               object
ncodpers                 int64
ind_empleado             int8
pais_residencia          int16
sexo                     int8
age                      int8
fecha_alta               object
ind_nuevo                int8
antiguedad               int32
indrel                   int8
ult_fec_cli_1t           object
indrel_1mes              int8
tiprel_1mes              int8
indresi                  int8
indext                   int8
conyuemp                 int8
canal_entrada            int16
indfall                  int8
tipodom                  int8
cod_prov                 int8
nomprov                  int8
ind_actividad_cliente    int8
renta                    int32
segmento                 int8
ind_ahor_fin_ult1        int8
ind_aval_fin_ult1        int8
ind_cco_fin_ult1         int8
ind_cder_fin_ult1        int8
ind_cno_fin_ult1         int8
ind_

In [15]:
df.isnull().any()

fecha_dato               False
ncodpers                 False
ind_empleado             False
pais_residencia          False
sexo                     False
age                      False
fecha_alta                True
ind_nuevo                False
antiguedad               False
indrel                   False
ult_fec_cli_1t            True
indrel_1mes              False
tiprel_1mes              False
indresi                  False
indext                   False
conyuemp                 False
canal_entrada            False
indfall                  False
tipodom                  False
cod_prov                 False
nomprov                  False
ind_actividad_cliente    False
renta                    False
segmento                 False
ind_ahor_fin_ult1        False
ind_aval_fin_ult1        False
ind_cco_fin_ult1         False
ind_cder_fin_ult1        False
ind_cno_fin_ult1         False
ind_ctju_fin_ult1        False
ind_ctma_fin_ult1        False
ind_ctop_fin_ult1        False
ind_ctpp

# 2. 피쳐 엔지니어링
- 의미없는 특성 삭제
- 파생변수 생성: 시간(년/월), Lag-1 데이타
- 특성선정, 따로 모아두기

In [0]:
# EDA를 통해 파악된 의미없는 특성은 지운다. 

# tipodom은 모두 동일한 값
# cod_prov 는 nomprov와 같은 의미
# conyuemp,ult_fec_cli_1t 두변수는 NA가 많음. 

df.drop(['tipodom', 'cod_prov', 
         'conyuemp', 'ult_fec_cli_1t'], axis=1, inplace= True)


In [0]:
#6 fecha_alta : 은행 첫 계약날짜 
#'2016-08-10' -> 2016, 8
# 연도와 월 정보를 분리하여 정수형으로 변환

df['fecha_alta_month']= df['fecha_alta'].map(lambda x:0.0 if x.__class__ is float 
                                             else float(x.split('-')[1])).astype(np.int8)
df['fecha_alta_year']= df['fecha_alta'].map(lambda x:0.0 if x.__class__ is float
                                            else float(x.split('-')[0])).astype(np.int16)
del df['fecha_alta']

In [0]:
# 생략
# #10 ult_fec_cli_1t : 1등급고객으로서 마지막날짜 
# # 연도와 월 정보를 분리하여 정수형으로 변환

# df['ult_fec_cli_1t_month']= df['ult_fec_cli_1t'].map(lambda x:0.0 if x.__class__ is float 
#                                              else float(x.split('-')[1])).astype(np.int8)
# df['ult_fec_cli_1t_year']= df['ult_fec_cli_1t'].map(lambda x:0.0 if x.__class__ is float 
#                                              else float(x.split('-')[0])).astype(np.int16)
# del df['ult_fec_cli_1t']

In [19]:
# 그 외 변수의 결측값은 모두 -99로 대체한다.
df.fillna(-99, inplace= True)
df.shape

(12020685, 45)

In [0]:
# lag-1 파생변수를 생성하기 위한 준비작업
# fecha_dato (고객정보수집)날짜를 숫자로 변환 (2015-01-28 ~ 2016-06-28) -> (1~18)

def date_to_int(str_date):
    Y, M, D= [int(a) for a in str_date.strip().split('-')]
    int_date= (int(Y)-2015)*12+int(M)
    return int_date

df['int_date']= df['fecha_dato'].map(date_to_int).astype(np.int8)

In [21]:
# lag-1 을 생성할 특성을 선정한다.

# 제외할 컬럼
# fecha_dato     날짜 2016-06-28
# sexo           성별 V,H
# antiguedad     은행 거래누적기간(월)
# indext         외국인지표(고객의 태어난 국가와 은행국가 동일여부, S/N)
# canal_entrada  고객유입채널(3, 4, KAT 등)
# fecha_alta_month 은행 첫 계약날짜 파생변수(월)
# fecha_alta_year 은행 첫 계약날짜 파생변수(월)

unselect_list= ['fecha_dato', 'sexo', 'antiguedad',
                'indext', 'canal_entrada',
                'fecha_alta_month', 'fecha_alta_year']

prev_list= []

for col in df.columns:
    if col not in unselect_list: 
        prev_list.append(col)

print(len(prev_list))
#prev_list
#39개

39


In [0]:
# lag-1 데이터 생성
# 데이터를 복사하고 변수명에 _prev추가
# int_date 날짜에 1을 더하여 lag를 생성한다. 

df_lag= df[prev_list].copy()
df_lag.columns= [col+'_prev' if col not in ['ncodpers', 'int_date']
                 else col for col in df[prev_list].columns]
df_lag['int_date'] += 1 

In [0]:
# 원본데이터와 lag데이터를 합친다. 
#lag데이터의 int_date는 한달 밀려있기때문에 지난달의 정보가 삽입된다

df_trn= df.merge(df_lag, on=['ncodpers', 'int_date'], how='left')

In [0]:
# import gc

# del [[df, df_lag]]
# gc.collect()
# df= pd.DataFrame()
# df_lag= pd.DataFrame()

In [0]:
del df
del df_lag

In [0]:
# 지난달 제품정보가 존재하지 않을경우는 0으로 대체한다.
for prod in prods:
    prev= prod + '_prev'
    df_trn[prev].fillna(0, inplace= True)

In [26]:
df_trn.fillna(-99, inplace= True) # Nan값이 많은 컬럼이 포함되어있으면 메모리부족으로 튕길수있다.
df_trn.shape

(12020685, 83)

In [0]:
# print(len(df_trn.columns.to_list()))
# df_trn.columns.to_list()
# 83개

In [27]:
# 모델학습에 사용할 피쳐들을 선정한다.

features=[]

# 제외할 컬럼
unselect_list= ['fecha_dato', 'ncodpers', 'int_date'] # index 컬럼
unselect_list += prods # 24개 상품보유컬럼

for col in df_trn.columns:
    if col not in unselect_list: 
        features.append(col)

print(len(features))
features

56


['ind_empleado',
 'pais_residencia',
 'sexo',
 'age',
 'ind_nuevo',
 'antiguedad',
 'indrel',
 'indrel_1mes',
 'tiprel_1mes',
 'indresi',
 'indext',
 'canal_entrada',
 'indfall',
 'nomprov',
 'ind_actividad_cliente',
 'renta',
 'segmento',
 'fecha_alta_month',
 'fecha_alta_year',
 'ind_empleado_prev',
 'pais_residencia_prev',
 'age_prev',
 'ind_nuevo_prev',
 'indrel_prev',
 'indrel_1mes_prev',
 'tiprel_1mes_prev',
 'indresi_prev',
 'indfall_prev',
 'nomprov_prev',
 'ind_actividad_cliente_prev',
 'renta_prev',
 'segmento_prev',
 'ind_ahor_fin_ult1_prev',
 'ind_aval_fin_ult1_prev',
 'ind_cco_fin_ult1_prev',
 'ind_cder_fin_ult1_prev',
 'ind_cno_fin_ult1_prev',
 'ind_ctju_fin_ult1_prev',
 'ind_ctma_fin_ult1_prev',
 'ind_ctop_fin_ult1_prev',
 'ind_ctpp_fin_ult1_prev',
 'ind_deco_fin_ult1_prev',
 'ind_deme_fin_ult1_prev',
 'ind_dela_fin_ult1_prev',
 'ind_ecue_fin_ult1_prev',
 'ind_fond_fin_ult1_prev',
 'ind_hip_fin_ult1_prev',
 'ind_plan_fin_ult1_prev',
 'ind_pres_fin_ult1_prev',
 'ind_reca_

In [0]:
df_trn.dtypes

fecha_dato                     object
ncodpers                        int64
ind_empleado                     int8
pais_residencia                 int16
sexo                             int8
age                              int8
ind_nuevo                        int8
antiguedad                      int32
indrel                           int8
indrel_1mes                      int8
tiprel_1mes                      int8
indresi                          int8
indext                           int8
canal_entrada                   int16
indfall                          int8
nomprov                          int8
ind_actividad_cliente            int8
renta                           int32
segmento                         int8
ind_ahor_fin_ult1                int8
ind_aval_fin_ult1                int8
ind_cco_fin_ult1                 int8
ind_cder_fin_ult1                int8
ind_cno_fin_ult1                 int8
ind_ctju_fin_ult1                int8
ind_ctma_fin_ult1                int8
ind_ctop_fin

# 4. 모델 Data 준비
- 전체 데이터셋에서 학습셋/검증셋/테스트셋을 구분한다.
- 입력값X와 타겟값Y을 구분한다.
    - 입력값X는 위에서 선정된 특성(features)이다.
    - 타겟값Y는 신규발생된 특정상품이다.
    
    
 - df_trn : trn /  tst 
    - trn -> XY :  XY_trn /  XY_vld
        - XY_trn : X_trn / Y_trn
        - XY_vld : X_vld / Y_vld
    - tst : X_tst


In [0]:
# 훈련데이터와 테스트데이터로 나눈다.

use_dates= ['2016-01-28', '2016-02-28', '2016-03-28', '2016-04-28', '2016-05-28']
trn = df_trn[df_trn['fecha_dato'].isin(use_dates)]
tst = df_trn[df_trn['fecha_dato'] == '2016-06-28'] # 테스트파일은 2016-06-28 밖에 없음.
del df_trn

In [29]:
print(trn.shape)
print(tst.shape)

(3462918, 83)
(929615, 83)


In [0]:
#  훈련데이터에서 신규구매건수만 추출한다.
X= []
Y= []
for i, prod in enumerate(prods):
    prev= prod + '_prev'
    prX= trn[(trn[prod] == 1) & (trn[prev] == 0)] # 신규발생
    prY= np.zeros(prX.shape[0], dtype=np.int8) + i # 상품순번 0~23
    X.append(prX) # 데이터프레임을 리스트에 담는다.
    Y.append(prY)
XY= pd.concat(X) # 리스트를 데이터프레임형태로 변환
Y= np.hstack(Y) # array의 차원을 줄인다. [[0],[1],[2,2,2],...] -> [0,1,2,2,2,....]
XY['y']= Y    

# 훈련셋, 검증셋으로 분리한다. (검증은 2016-05-28 을 사용)

vld_date= '2016-05-28'
XY_trn= XY[XY['fecha_dato'] != vld_date]
XY_vld= XY[XY['fecha_dato'] == vld_date]

In [31]:
print(XY.shape)
print(XY_trn.shape)
print(XY_vld.shape)

(199034, 84)
(161137, 84)
(37897, 84)


In [0]:
pd.set_option('max_columns', 5)
XY.head()

,fecha_dato,ncodpers,...,ind_recibo_ult1_prev,y
10597872,2016-05-28,194160,...,0.0,0
7658069,2016-01-28,1474324,...,1.0,1
7628180,2016-01-28,1432311,...,0.0,2
7628198,2016-01-28,1432232,...,0.0,2
7628482,2016-01-28,1432080,...,0.0,2


#5. 모델 학습

- page 81,  파라메타 설명
- 하이퍼파라메타 튜닝  < 피쳐엔지니어링

- df_trn : trn /  tst <br>
    - trn -> XY :  XY_trn /  XY_vld
        - XY_trn : X_trn / Y_trn
        - XY_vld : X_vld / Y_vld
    - tst : X_tst

In [32]:
import xgboost as xgb

# xgboost 파라메타 설정
param={
    'booster': 'gbtree',
    'max_depth':8,
    'nthread':4,
    'num_class':len(prods), # 클래스 설정
    'objective': 'multi:softprob',
    'silent':1,
    'eval_metric': 'mlogloss',
    'eta': 0.1,
    'min_child_weight': 10,
    'colsample_bytree': 0.8,
    'colsample_bylevel': 0.9,
    'seed': 2018
}

# 훈련, 검증데이터를  xgboost에 사용가능한 형태로 변환
# 훈련, 검증 데이터에서 사용할 feature만 추출하고 x,y를 분리한다.
# (인덱스와 컬럼이 없는) 값만 추출한 행렬형태로 변환한다.

X_trn= XY_trn.as_matrix(columns= features) # XY_trn[features].values
Y_trn= XY_trn.as_matrix(columns= ['y'])
dtrn= xgb.DMatrix(X_trn, label= Y_trn, feature_names= features)
X_vld= XY_vld.as_matrix(columns= features)
Y_vld= XY_vld.as_matrix(columns= ['y'])
dvld= xgb.DMatrix(X_vld, label= Y_vld, feature_names= features)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:23: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:24: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:26: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:27: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.


In [33]:
print('X입력값 형태\n', X_trn.shape)
print('Y목표값 형태\n', Y_trn.shape)
print('첫째건 x\n', X_trn[0])
print('첫째건 y\n', Y_trn[0])

X입력값 형태
 (161137, 56)
Y목표값 형태
 (161137, 1)
첫째건 x
 [ 0.000e+00  0.000e+00  1.000e+00  4.300e+01  1.000e+00  3.000e+00
  1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00  3.000e+00
  0.000e+00  1.800e+01  1.000e+00 -9.900e+01  2.000e+00  1.000e+01
  2.015e+03  0.000e+00  0.000e+00  4.300e+01  1.000e+00  1.000e+00
  1.000e+00  0.000e+00  0.000e+00  0.000e+00  1.800e+01  1.000e+00
 -9.900e+01  2.000e+00  0.000e+00  0.000e+00  1.000e+00  0.000e+00
  0.000e+00  0.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00
  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00  0.000e+00
  0.000e+00  1.000e+00  1.000e+00  0.000e+00  0.000e+00  0.000e+00
  0.000e+00  1.000e+00]
첫째건 y
 [1]


In [0]:
# 모델학습 # 30분정도 걸린다.
# 다른모델로 돌려보고 비교해보자.
watch_list= [(dtrn, 'train'), (dvld, 'eval')]
model= xgb.train(param, dtrn, num_boost_round=1000, evals= watch_list,
                 early_stopping_rounds=20)
# mlogloss? multiclass logloss -> categorical cross entropy -> https://pythonkim.tistory.com/20

In [0]:
# 학습한 모델을 저장한다.

import pickle

#pickle.dump(model, open("./kaggle_data/xgb.baseline.pkl", "wb") # 학습한 모델을 저장
model = pickle.load(open("./kaggle_data/xgb.baseline.pkl", "rb")) # 학습한 모델을 로드

best_ntree_limit= model.best_ntree_limit


#6. 모델평가(검증)
- 교차검증
    - MAP@7 평가함수 (Mean Average Precision @ 7), page50
        - Predict [2,12,10,9,1,0,8], True [2,11,8,9,1,0,7]
        - 1 0 0 1 1 1 0 -> 정확도: (1/1 + 0 + 0 + 2/4 + 3/5 + 4/6 + 0)/4 =  0.69
        - Predict [2,12,10,9,1,0,8], True [2,12,10,9,3,4,7]
        - 1 1 1 1 0 0 0 -> 정확도: (1/1 + 2/2 + 3/3 + 4/4 + 0 + 0 + 0)/4 =  1
        - Predict [5,11,9,10,3,4,7], True [2,12,8,10,3,4,7]
        - 0 0 0 1 1 1 1 -> 정확도: (0 + 0 + 0 + 1/4 + 2/5 + 3/6 + 4/7)/4 =  0.43
        - 예측의 순서에 매우 예민한 평가척도이다.
        
    - 검증셋에 대해서 정답 점수 vs 예측 점수를 비교한다.

In [0]:
# MAP@7 평가함수

def apk(actual, predicted, k=7, default= 0.0):
    if len(predicted) > k:
        predicted = predicted[:k]
        
    score= 0.0
    num_hits= 0.0
    
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
            
            
    if not actual:
        return default
    
    return score / min(len(actual), k)


def mapk(actual, predicted, k=7, default=0.0):
    return np.mean([apk(a, p, k, default) for a, p, in zip(actual, predicted)])
    

In [39]:
# 검증데이터에 대해 MAP@7 평가한다.
# 검증데이터의 실제 정답값을 기반으로 평가했을때 점수는?

vld_date= '2016-05-28'
vld= trn[trn['fecha_dato'] == vld_date] # 고객식별번호 추출
ncodpers_vld= vld.as_matrix(columns=['ncodpers'])

# 검증데이터에서 신규구매를 구한다.
for prod in prods:
    prev= prod + '_prev'
    padd= prod + '_add'
    vld[padd]= vld[prod] - vld[prev]
add_vld= vld.as_matrix(columns=[prod+'_add' for prod in prods]) # [[0,0,0,.....],[0,0,1,0,0,0...]....]
add_vld_list= [list() for i in range(len(ncodpers_vld))] # 빈리스트 생성

# 고객별 신규구매 정답값을 add_vld_list에 저장, 총 count를 count_vld에 저장
count_vld= 0
for ncodper in range(len(ncodpers_vld)):
    for prod in range(len(prods)):
        if add_vld[ncodper, prod] > 0: # 신규구매 +1일때
            add_vld_list[ncodper].append(prod) # 해당제품의 순번(0~23) [[],[3]...[2,23],...]
            count_vld += 1 # 해당 변수는 사용되지 않는다?

# 검증데이터의 MAP7 최고점을 구한다. 0.04266
# page 83, 검증데이터의 실제 정답값을 기반으로 했을때 점수가 1이 나오지 않는이유는?
print(mapk(add_vld_list, add_vld_list, 7, 0.0))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Remove the CWD from sys.path while we load stuff.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:11: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  # This is added back by InteractiveShellApp.init_path()


0.04266379915553903


In [40]:
# 검증데이터에 대해 MAP@7 평가한다.
# 모델예측 값은?

X_vld= vld.as_matrix(columns= features)
Y_vld= vld.as_matrix(columns= ['y']) # 빈매트릭스, nan값이 생성
dvld= xgb.DMatrix(X_vld, label= Y_vld, feature_names= features)

# 검증데이터에 대한 예측값을 구한다.
# preds_vld 는 (696539, 24) 으로 각 상품에 대해 확률값을 반환한다.
preds_vld= model.predict(dvld, ntree_limit= best_ntree_limit)

# 지난달에 보유한 상품은 신규구매가 불가하기 때문에 확률값에서 미리 1을 빼준다.
# 예측상품이 이전상품과 중복될때는 0으로 변환(신규구매에대한 예측이므로)
preds_vld= preds_vld - vld.as_matrix(columns= [prod+'_prev' for prod in prods]) 


# 검증 데이터 예측 상위 7개를 추출한다.
result_vld= []
for ncodper, pred in zip(ncodpers_vld, preds_vld):
    y_prods= [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods= sorted(y_prods, key=lambda a: a[0], reverse= True)[:7] # 확률값이 가장 높은 순서로 7순위정렬
    result_vld.append([ip for y,p,ip in y_prods])
    
# 검증데이터의 MAP7을 구한다. 0.03646
print(mapk(add_vld_list, result_vld, 7, 0.0))

# 약 85%의 정확도를 보여준다.
# 0.03647/0.04266

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


ValueError: ignored

In [0]:
vld.head()

,fecha_dato,ncodpers,...,ind_nom_pens_ult1_add,ind_recibo_ult1_add
10394531,2016-05-28,657788,...,0.0,0.0
10394532,2016-05-28,657795,...,0.0,0.0
10394533,2016-05-28,657790,...,0.0,0.0
10394534,2016-05-28,657794,...,0.0,0.0
10394535,2016-05-28,657789,...,0.0,0.0


In [0]:
preds_vld[-1]

array([1.00886507e-06, 6.34939033e-06, 7.09518135e-01, 3.14121498e-05,
       2.26990730e-02, 7.70537517e-05, 1.59127280e-01, 2.10248181e-05,
       1.50917640e-05, 6.34938442e-06, 1.00886507e-06, 5.23785595e-04,
       3.64034460e-03, 2.61658744e-04, 2.77242216e-05, 9.06465066e-05,
       2.01313946e-04, 6.18559076e-04, 2.57772161e-04, 5.12752391e-04,
       2.07267531e-05, 3.32006440e-02, 3.56455930e-02, 3.34946699e-02])

In [0]:
y_prods

[(0.7095181345939636, 'ind_cco_fin_ult1', 2),
 (0.15912728011608124, 'ind_ctma_fin_ult1', 6),
 (0.03564559295773506, 'ind_nom_pens_ult1', 22),
 (0.03349466994404793, 'ind_recibo_ult1', 23),
 (0.033200643956661224, 'ind_nomina_ult1', 21),
 (0.02269907295703888, 'ind_cno_fin_ult1', 4),
 (0.00364034459926188, 'ind_ecue_fin_ult1', 12)]

In [0]:
add_vld_list[0:3]

[[], [], []]

In [0]:
result_vld[0:3]

[[23, 18, 12, 22, 21, 4, 11],
 [23, 18, 12, 21, 22, 4, 19],
 [19, 17, 8, 7, 2, 13, 11]]

In [0]:
# 특성중요도 확인
for kv in sorted([(k,v) for k,v in model.get_fscore().items()], 
                 key= lambda kv: kv[1], reverse= True):
    print(kv)


('renta', 28920)
('antiguedad', 27816)
('age', 21539)
('age_prev', 15253)
('fecha_alta_month', 14544)
('renta_prev', 13650)
('nomprov', 12703)
('canal_entrada', 10709)
('fecha_alta_year', 10175)
('nomprov_prev', 7096)
('sexo', 4067)
('ind_recibo_ult1_prev', 3469)
('ind_cco_fin_ult1_prev', 3236)
('ind_ecue_fin_ult1_prev', 3137)
('ind_cno_fin_ult1_prev', 2973)
('segmento', 2619)
('segmento_prev', 2256)
('ind_tjcr_fin_ult1_prev', 2146)
('ind_reca_fin_ult1_prev', 2030)
('ind_nom_pens_ult1_prev', 1671)
('ind_dela_fin_ult1_prev', 1600)
('ind_ctop_fin_ult1_prev', 1553)
('ind_valo_fin_ult1_prev', 1552)
('tiprel_1mes', 1478)
('ind_nomina_ult1_prev', 1361)
('ind_actividad_cliente', 1281)
('ind_ctpp_fin_ult1_prev', 1139)
('indext', 1095)
('ind_fond_fin_ult1_prev', 1062)
('ind_actividad_cliente_prev', 1038)
('tiprel_1mes_prev', 1018)
('ind_ctma_fin_ult1_prev', 936)
('ind_plan_fin_ult1_prev', 724)
('indrel_1mes', 701)
('ind_hip_fin_ult1_prev', 536)
('ind_nuevo_prev', 532)
('ind_nuevo', 514)
('indre

# 6. 모델 재학습 및 테스트셋 예측

In [0]:
# 모델 재학습
X_all= XY.as_matrix(columns= features)
Y_all= XY.as_matrix(columns= ['y'])
dall= xgb.DMatrix(X_all, label= Y_all, feature_names= features)
watch_list=[(dall, 'train')]
best_ntree_limit= int(best_ntree_limit * (len(XY_trn) + len(XY_vld)) / len(XY_trn))
model= xgb.train(param, dall, num_boost_round= best_ntree_limit, evals= watch_list)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


[0]	train-mlogloss:2.70808
[1]	train-mlogloss:2.47015
[2]	train-mlogloss:2.28692
[3]	train-mlogloss:2.14514
[4]	train-mlogloss:2.03119
[5]	train-mlogloss:1.94119
[6]	train-mlogloss:1.86354
[7]	train-mlogloss:1.79325
[8]	train-mlogloss:1.73147
[9]	train-mlogloss:1.67908
[10]	train-mlogloss:1.63023
[11]	train-mlogloss:1.58655
[12]	train-mlogloss:1.54865
[13]	train-mlogloss:1.51247
[14]	train-mlogloss:1.47985
[15]	train-mlogloss:1.44949
[16]	train-mlogloss:1.42189
[17]	train-mlogloss:1.39823
[18]	train-mlogloss:1.37597
[19]	train-mlogloss:1.35466
[20]	train-mlogloss:1.33545
[21]	train-mlogloss:1.31822
[22]	train-mlogloss:1.30198
[23]	train-mlogloss:1.28682
[24]	train-mlogloss:1.27257
[25]	train-mlogloss:1.25928
[26]	train-mlogloss:1.24711
[27]	train-mlogloss:1.23613
[28]	train-mlogloss:1.22574
[29]	train-mlogloss:1.21595
[30]	train-mlogloss:1.20711
[31]	train-mlogloss:1.19871
[32]	train-mlogloss:1.19069
[33]	train-mlogloss:1.18295
[34]	train-mlogloss:1.17576
[35]	train-mlogloss:1.16905
[3

In [0]:
# 캐글 제출을 위하여 테스트 데이터에 대한 예측값을 구한다.

x_tst= tst.as_matrix(columns= features)
dtst= xgb.DMatrix(x_tst, feature_names= features)
preds_tst= model.predict(dtst, ntree_limit= best_ntree_limit)
ncodpers_tst= tst.as_matrix(columns=['ncodpers'])
preds_tst= preds_tst - tst.as_matrix(columns= [prod + '_prev' for prod in prods])


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:4: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  after removing the cwd from sys.path.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """


In [0]:
preds_tst

array([[ 5.6745657e-06,  5.6745657e-06, -9.9996901e-01, ...,
         1.3862401e-02,  1.8687792e-02,  8.2583559e-01],
       [ 3.5360063e-06,  3.5360063e-06, -9.9997342e-01, ...,
         4.1360021e-02,  4.3589003e-02,  8.0821025e-01],
       [ 3.5199002e-06,  3.5199002e-06, -9.9997133e-01, ...,
         3.1809099e-02,  3.2715838e-02,  8.9156580e-01],
       ...,
       [ 5.9086483e-06,  5.9086483e-06, -9.9993151e-01, ...,
         3.2171752e-02,  2.2744672e-02,  5.0091499e-01],
       [ 3.1719446e-06,  3.1719446e-06, -9.9997580e-01, ...,
         7.8792647e-03,  1.1644844e-02,  7.0173132e-01],
       [ 4.7555432e-06,  4.7555432e-06,  7.6168704e-01, ...,
         4.6343966e-03,  1.5641242e-02,  8.0171317e-02]], dtype=float32)

In [0]:
preds_tst.shape
# (929615, 24)

(929615, 24)

In [0]:
# 제출파일을 생성한다.

submit_file= open('./kaggle_data/xgb_baseline.1', 'w')
submit_file.write('ncodpers,added_products\n')

for ncodper, pred in zip(ncodpers_tst, preds_tst):
    y_prods= [(y,p,ip) for y,p,ip in zip(pred, prods, range(len(prods)))]
    y_prods= sorted(y_prods, key=lambda a: a[0], reverse= True)[:7]
    y_prods= [p for y,p,ip in y_prods]
    submit_file.write('{},{}\n'.format(int(ncodper), ' '.join(y_prods)))


## baseline model 결과
__________________________
- Private Score 0.02430
- Public Score 0.02396



In [0]:
# 다른 모델로 돌려보기
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier

In [0]:
from sklearn import metrics #accuracy measure
from sklearn.metrics import confusion_matrix #for confusion matrix

from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold #for K-fold cross validation
from sklearn.model_selection import cross_val_score #score evaluation
from sklearn.model_selection import cross_val_predict #prediction

In [0]:
from sklearn.model_selection import StratifiedKFold, cross_val_score

def scoring(model, predictors, target):
    kf = StratifiedKFold(n_splits=5, random_state=0, shuffle=False)
    score = cross_val_score(model, predictors, target, cv=kf, scoring='f1_macro').mean()
    return score



In [0]:
X_all= XY.as_matrix(columns= features)
Y_all= XY.as_matrix(columns= ['y'])

rf= RandomForestClassifier(n_estimators=100, random_state=0)
acc_rf = scoring(rf, X_all, Y_all) 
print(acc_rf)



/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437

0.26702919750217124


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


0.24634114391515505

In [0]:
import xgboost as xgb
xg= xgb.XGBClassifier(booster='gbtree', max_depth=8, nthread=4,
                 objective='multi:softprob', silent=1, eta=0.1,
                 min_child_weight=10, colsample_bytree=0.8,
                 colsample_bylevel=0.9, seed=2018)
xg.fit(X_trn,Y_trn)


/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:219: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/label.py:252: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=0.9,
              colsample_bynode=1, colsample_bytree=0.8, eta=0.1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=8,
              min_child_weight=10, missing=None, n_estimators=100, n_jobs=1,
              nthread=4, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=2018,
              silent=1, subsample=1, verbosity=1)

In [0]:
print('훈련세트 정확도:{:.3f}'.format(xg.score(X_trn, Y_trn)))
print('검증세트 정확도:{:.3f}'.format(xg.score(X_vld, Y_vld)))
# score는 정확도를 나타낸다.
# 정확도 = TP+TN / TP+TN+FP+FN 으로 정확히 예측한수를 전체 샘플수로 나눈것.


훈련세트 정확도:0.619
검증세트 정확도:0.601


In [0]:
Y_vld.reshape(-1)[5000:5100]

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2], dtype=int8)

In [0]:
pred[5000:5100]

array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 12,  2,  2,
        2,  2,  2,  2,  2,  2, 23,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2, 18,  2,  2,  2,  2,  2,  2, 23, 22,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 22,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2, 21,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2],
      dtype=int8)

In [0]:
ada=AdaBoostClassifier(n_estimators=200, learning_rate=0.05, random_state=0)
ada.fit(X_trn,Y_trn)
print('훈련세트 정확도:{:.3f}'.format(ada.score(X_trn, Y_trn)))
print('검증세트 정확도:{:.3f}'.format(ada.score(X_vld, Y_vld)))
# 상기동일
# pred= ada.predict(X_trn)
# print('훈련세트 정확도:{:.3f}'.format(metrics.accuracy_score(pred,Y_trn)))

# 훈련세트 정확도:0.560
# 훈련세트 정확도:0.550


/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:724: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=0.1,
                   n_estimators=200, random_state=0)

In [0]:
rf= RandomForestClassifier(n_estimators=150, random_state=0)
rf.fit(X_trn,Y_trn)
print('훈련세트 정확도:{:.3f}'.format(rf.score(X_trn, Y_trn)))
print('검증세트 정확도:{:.3f}'.format(rf.score(X_vld, Y_vld)))

# 훈련세트 정확도:0.794
# 검증세트 정확도:0.609

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  


훈련세트 정확도:0.794
검증세트 정확도:0.609


In [0]:
n_estimators=range(50,200,50)
hyper={'n_estimators':n_estimators}
gd=GridSearchCV(estimator=RandomForestClassifier(random_state=0),param_grid=hyper,verbose=True)
gd.fit(X_trn, Y_trn)
print(gd.best_score_)
print(gd.best_estimator_)

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:1978: FutureWarning: The default value of cv will change from 3 to 5 in version 0.22. Specify it explicitly to silence this warning.
  warnings.warn(CV_WARNING, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=3.
  % (min_groups, self.n_splits)), Warning)
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)


Fitting 3 folds for each of 3 candidates, totalling 9 fits


/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  estimator.fit(X_train, y_train, **fit_params)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_validation.py:514: DataConversionWarning: A column-vector y was passed when a 

0.5732265091195691
RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
                       max_depth=None, max_features='auto', max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=0, verbose=0,
                       warm_start=False)
